# GPS

Based on D. Kalman's method, I expand the four constraint equations as following:

$
\begin{align}
2.4x + 4.6y + 0.4z - 2(0.047^2)9.9999 t &= 1.2^2 + 2.3^2 + 0.2^2 + x^2 + y^2 + z^2 - 0.047^2 t^2 \\
-1x + 3y + 3.6z - 2(0.047^2)13.0681 t &= 0.5^2 + 1.5^2 + 1.8^2 +  x^2 + y^2 + z^2 - 0.047^2 t^2 \\
-3.4x + 1.6y + 2.6z - 2(0.047^2)2.0251 t &= 1.7^2 + 0.8^2 + 1.3^2 +  x^2 + y^2 + z^2 - 0.047^2 t^2 \\
3.4x + 2.8y - 1z - 2(0.047^2)10.5317 t &= 1.7^2 + 1.4^2 + 0.5^2 +  x^2 + y^2 + z^2 - 0.047^2 t^2 
\end{align},
$

And subtract the first equation from the rest.

$
\begin{align}
-3.4x - 1.6y + 3.2z - 2(0.047^2)3.0682 t &= -1.0299  \\
-5.8x - 3y + 2.2z + 2(0.047^2)7.9750 t &= -1.5499  \\
1x    - 1.8y - 1.4z - 2(0.047^2)0.5318 t &= -1.6699
\end{align},
$

Converting it to matrix format as following:

$
\begin{equation}
\begin{bmatrix}
-3.4 & 1.6 & 3.2 & -0.01355  \\
-5.8 & -3   & 2.2 & 0.0352   \\
1    & -1.8 & -1.4 & -0.00235 
\end{bmatrix}
\begin{bmatrix}
x \\ y \\ z \\ t
\end{bmatrix}
=
\begin{bmatrix}
-1.0299 \\ -1.5499 \\ -1.6699
\end{bmatrix}
\end{equation}.
$

# Newton's Method

For having $f(x)=0$, based on state variables $x=(x, y, z, t)$ we can convert the nonlinear distance equation as following:

$
\begin{equation}
f(x) = \frac{1}{2}
\begin{bmatrix}
(x-1.2)^2+(y-2.3)^2+(z-0.2)^2-(0.047*(t-09.9999))^2 \\
(x+0.5)^2+(y-1.5)^2+(z-1.8)^2-(0.047*(t-13.0681))^2 \\
(x+1.7)^2+(y-0.8)^2+(z-1.3)^2-(0.047*(t-02.0251))^2 \\
(x-1.7)^2+(y-1.4)^2+(z+0.5)^2-(0.047*(t-10.5317))^2
\end{bmatrix}
= 0.
\end{equation}
$

In [1]:
f(x) = [(x[1]-1.2)^2+(x[2]-2.3)^2+(x[3]-0.2)^2-0.047*(x[4]-09.9999)^2,
    (x[1]+0.5)^2+(x[2]-1.5)^2+(x[3]-1.8)^2-0.047*(x[4]-13.0681)^2,
    (x[1]+1.7)^2+(x[2]-0.8)^2+(x[3]-1.3)^2-0.047*(x[4]-02.0251)^2,
    (x[1]-1.7)^2+(x[2]-1.4)^2+(x[3]+0.5)^2-0.047*(x[4]-10.5317)^2]./2

f([1.2 2.3 0.2 9.9999])

4-element Array{Float64,1}:
 0.0
 2.82377449586
 4.4404602765600005
 0.7683539358599999

## Jacobian

$
\begin{equation}
Df(x_0) = 
\begin{bmatrix}
(x-1.2) & (y-2.3) & (z-0.2) & -0.047(t-09.9999)\\
(x+0.5) & (y-1.5) & (z-1.8) & -0.047(t-13.0681)\\
(x+1.7) & (y-0.8) & (z-1.3) & -0.047(t-02.0251)\\
(x-1.7) & (y-1.4) & (z+0.5) & -0.047(t-10.5317)
\end{bmatrix}.
\end{equation}
$

In [2]:
D(x) = [x[1]-1.2 x[2]-2.3 x[3]-0.2 -0.047*(x[4]-09.9999);
    x[1]+0.5 x[2]-1.5 x[3]-1.8 -0.047*(x[4]-13.0681);
    x[1]+1.7 x[2]-0.8 x[3]-1.3 -0.047*(x[4]-02.0251);
    x[1]-1.7 x[2]-1.4 x[3]+0.5 -0.047*(x[4]-10.5317)]/2

D([1.7 1.4 -0.5 10.5317])

4×4 Array{Float64,2}:
 0.25  -0.45  -0.35  -0.0124973
 1.1   -0.05  -1.15   0.0596054
 1.7    0.3   -0.9   -0.199905
 0.0    0.0    0.0   -0.0

# Solutions

In [3]:
using LinearAlgebra

function gps(x)
    t = 1            # tolerance of answers
    for i=1:1e5
        d = f(x)\D(x)
        x -= d
        if norm(d)<1e-15 return x end
    end
    return x
end

s = gps([0 0 0 0])

1×4 Array{Float64,2}:
 -0.152014  1.16284  0.239069  3.61735

In [4]:
b = [-1.0299;-1.5499;-1.6699]
A = [-3.4  1.6  3.2  -0.01355  ;
-5.8  -3    2.2  0.0352   ;
1     -1.8  -1.4  -0.00235] 

s = b\A

1×4 Transpose{Float64,Array{Float64,1}}:
 1.73099  0.961006  -0.698654  -0.00586697

## Adjourn

In [5]:
using Dates

println("mahdiar")
Dates.format(now(), "Y/U/d HH:MM")  

mahdiar


"2021/March/19 11:13"